In [1]:
import tree as a
import os

#保存到文件的函数
def Save2File(data):
    '''
    把数据集保存到文件
    :param data:
    :return:
    '''
    path = os.path.abspath('..')  # 表示当前所处的文件夹上一级文件夹
    data_path = path + '\\data\\output.txt'
    data_file = open(data_path, 'w')
    str=''
    for each in data:
        str=str + '{}\n'.format(each)
    data_file.write(str)
    data_file.close()


#------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------
#生成树的函数
def GetTrees():
    '''
    获取根据属性生成所有树
    :return: 返回所有属性的树，顺序在'attribute.txt'里面
    '''
    attribute = []
    path = os.path.abspath('..')  # 表示当前所处的文件夹上一级文件夹的绝对路径
    path1 = path + "\data//attribute"
    infile = open(path1, 'r')
    attribute_file = infile.readline()[:-1]  # 读取一行
    attribute = (attribute_file.split(','))
    trees = []
    for i in range(len(attribute)):
        path2 = path + "\data//attribute_" + attribute[i]
        tree = a.Tree1()
        tree1 = tree.createTree(path2)
        trees.append(tree1)
    return trees

#生成树的函数
def GetTreesDict():
    '''
    获取根据属性生成所有树
    :return: 返回所有属性的树，顺序在'attribute.txt'里面
    '''


    treeDict ={}

    path = os.path.abspath('..')  # 表示当前所处的文件夹上一级文件夹的绝对路径
    path1 = path + "\data//attribute"
    infile = open(path1, 'r')
    attribute_file = infile.readline()[:-1]  # 读取一行
    attribute = (attribute_file.split(','))
    trees = []
    for i in range(len(attribute)):
        path2 = path + "\data//attribute_" + attribute[i]
        tree = a.Tree1()
        tree1 = tree.createTree(path2)
        treeDict[attribute[i]] = tree1
    return treeDict

def Test_K_Anonymity(lowLoss_data,QID,K):
    KN_result = []
    count = 0
    for i in range(len(lowLoss_data)):
        tmp_k = 1
        same = False
        for j in range(len(lowLoss_data)):
            if i == j:
                continue
            if Equal(lowLoss_data.iloc[i], lowLoss_data.iloc[j,:], QID):
                tmp_k = tmp_k + 1
                if tmp_k >= K:
                    same = True
                    break
        count = count + 1
        # print("比对数据中..." + str(count))
        KN_result.append(same)



    return KN_result


#------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------
#获取PayOff的函数
def GetPayOff(tree,attribute):
    '''
    获取非数字型的PayOff
    :param tree: 属性的树
    :param attribute: 属性值
    :return: PayOff
    '''
    node = tree.get_node(attribute)
    level_all = tree.depth() + 1   #树的总层数
    level_site = tree.depth() + 1 - tree.depth(node)   #当前节点所在层数
    return float(level_site/level_all)   #返回当前节点的payoff

#------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------
#判断两个列表是否相等
def Equal(a,b,QID):
    '''
    判断两个列表对应的属性是否相等
    :param a: 列表a
    :param b: 列表b
    :param QID: 属性位置
    :return: 对应属性相同，返回True,不相同返回False
    '''
    for i in QID:
        if a[i]!=b[i]:
            return False

    return True

#-----------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------
'''
下面一个函数属性网上匿名的函数
'''
def climb(tree,attribute):
    '''
    属性进一步根据树匿名
    :param tree: 属性的树
    :param attribute: 属性值
    :return: 进一步匿名后的结果
    '''
    node = tree.get_node(attribute)
    return (tree.parent(attribute).tag)   #返回父节点


def generalize(tree_dict,gen_col, data, index_boolean):
    '''
    本函数根据泛化树，对数据的指定属性的指定样本进行泛化，并返回泛化后的结果
    :param tree:
    :param attribute:
    :param data:
    :param index:
    :return:
    '''

    tree = tree_dict.get(gen_col)

    for i in range(len(data)):

        flag = index_boolean[i]
        if flag:
            tmp_attribute = climb(tree, data.loc[i][gen_col])  # 得到父节点
            data.loc[i][gen_col] = tmp_attribute  # 更新匿名属性


    return data

def final_generalize(tree_dict,col_list, data):
    '''
    本函数根据泛化树，对数据的指定属性的指定样本进行泛化，并返回泛化后的结果
    :param tree:
    :param attribute:
    :param data:
    :param index:
    :return:
    '''

    for gen_col in col_list:
        tree = tree_dict.get(gen_col)
        for i in range(len(data)):

            tmp_attribute = tree.root  # 得到根节点，直接泛化
            data.loc[i][gen_col] = tmp_attribute  # 更新匿名属性


    return data
#----------------------------------------------------------------------------
#----------------------------------------------------------------------------
#----------------------------------------------------------------------------
'''
以下四个函数是获取损失率的函数
'''
def GetLoss(tree,attribute):
    '''
    根据是否是数值型的树，返回损失值
    :param tree: 属性的树
    :param attribute: 属性值
    :return: 损失率
    '''
    #job_tree Lawyer
    if(IntTree(tree)==True):
        return GetNumLoss(tree,attribute)

    else:
        return GetCharLoss(tree,attribute)



def IntTree(tree):
    '''
    判断是否是数值型的树，是返回True，不是返回false
    :param tree: 属性的树
    :return: true或者false
    '''
    node = tree.nodes.keys()
    keys=[]
    for each in node:
        keys.append(each)
    if '-' in keys[0]:
        return True
    else:
        return False


def GetCharLoss(tree,str):
    '''
    获取属性地损失
    :param tree: 属性树
    :param str: 属性值
    :return: 损失
    '''

    Child_len = len(tree.children(str))  # 子节点的个数 Lawyer 没有children
    Child_all = Child_len
    if Child_len == 0:  # 还未泛化的最底层节点
        return 0 #Lawyer 是最底层节点，为0

    node = tree.get_node(str)
    level_site = tree.depth(node)  # 获取当前节点的深度
    if (level_site == 0):  # 根节点的深度为0
        return 1

    parent = tree.parent(str).tag  # 返回父节点
    childrens = tree.children(parent)  # 获取父节点
    for i in range(len(childrens)):  # 获取下一层的所有子节点
        if childrens[i].tag != str:
            parent1 = childrens[i].tag
            Child_all = Child_all + len(tree.children(parent1))  # 依次累加
    return (Child_len / Child_all)   #返回非数值型的loss



def GetNumLoss(tree,age):
    '''
    获取数值型的PayOff
    :param tree: 属性地树
    :param age: 属性值
    :return: PayOff
    '''
    Numrange = []
    depth = tree.depth()   #树的深度
    node = tree.get_node(age)    #当前节点所在层数
    level_site = tree.depth(node)  # 获取当前节点的深度
    if (level_site == 0):  # 根节点的深度为0
        return 1

    if (level_site == depth):   #未泛化的最底层节点
        return 0

    child = age.split('-')
    Range = int(child[1]) - int(child[0])   #当前节点的范围
    parent = tree.parent(age).tag    #获取父节点
    Numrange = parent.split('-')
    Range1 = int(Numrange[1]) - int(Numrange[0])     #父节点的范围
    return(Range/Range1)    #返回数值型的loss

def Remove(data,EQ,QID ):
    remove_set=[]
    for i in range(len(data)):
        each = data[i]
        if Equal(EQ,each,QID):
            remove_set.append(i)
    remove_set.sort(reverse=True)
    for j in remove_set:
        data.remove(data[j])
    return data


In [2]:
cls = ['age', 'workclass', 'fnlwgt',  'education', 'education-num', 'marital-status', 'occupation', 'relationship', 
       'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label']  # 数据属性

from time import sleep
import math
import os
import pandas as pd
data = []
Trees = GetTrees()   #保存属性的树
TreeDict = GetTreesDict()
k = 2

In [3]:
#第一步：读取数据，生成元组
path = os.path.abspath('..')  # 表示当前所处的文件夹上一级文件夹
data_path = path + '\\data\\data.csv'
data_file = open(data_path, 'r')
lines = data_file.readlines()
for line in lines:
    i = line[:-1].split(',')
    data.append(i)


In [4]:
cls = ['age', 'workclass', 'fnlwgt',  'education', 'education-num', 'marital-status', 'occupation', 'relationship', 
       'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label']  # 数据属性
QID = ['age', 'workclass','education','marital-status','race','sex','native-country']

## 用于存储最终结果
result =pd.DataFrame(columns=QID)
precision = 0


In [5]:
if k < 2:
    print("参数k不能小于2")
    exit()

data_len = len(data)

#【2】初始数据集如果小于K，则直接退出
if data_len < k:
    print("数据总量小于匿名数量，无法满足条件")
    exit()

In [6]:
## 转换成pandas数据类型进行处理
raw_data = pd.DataFrame(data, columns=QID)

In [7]:
tmp_data = raw_data.copy()

In [8]:

# while True:
#     【1】 对原始数据进行拷贝后，进行K匿名检测
k_check_res = Test_K_Anonymity(tmp_data,QID,k)
k_sum = sum(k_check_res) ## 计算tmp_data 中满足 k 匿名的数据条数
total_items = tmp_data.shape[0]
if k_sum == total_items:
    ## 满足K匿名条件, 将数据加入到最终结果集，并停止运行
    print("满足最终K匿名，停止运算")
    result.append(tmp_data)
#     break
elif total_items <= k: ## 数据总条数是否小于等于 K值，若是则全部泛化为最高层级，并加入最终结果集
    print("数据条数小于K，则进行最高级泛化")
    gen_data = final_generalize(TreeDict, tmp_data.columns, tmp_data)
    tmp_res = result.copy().append(gen_data) ## 将泛化后的数据，加入到最终结果集，并判断是否满足K匿名
    tmp_check_res = Test_K_Anonymity(tmp_res,QID,k)
    tmp_sum = sum(tmp_check_res)
    if tmp_sum == tmp_res.shape[0]: ## 若满足K匿名条件，则输出最终结果，否则不做任何处理
        result = tmp_res
        print("剩余数据进行最高级泛化后，加入最终结果集，仍然满足K匿名条件，停止运算")
    else:
        print("剩余数据进行最高级泛化后，加入最终结果集,无法满足K匿名条件，停止运算")
#     break
else: ## 数据条数> K， 则将满足K匿名的数据条目全部抽取到最终结果后，执行第三步
    print("数据条数 > K， 将数据集中的，满足K匿名的样本，提取到最终结果集中")
    result.append(tmp_data.loc[k_check_res])
    
    tmp_data = tmp_data[~k_check_res] ## 提取后，获得提取后的数据
    
    '''
    第三步，属性集长度为n，选择n-1个属性组成n种集合的元组，统计出各元组中存在的等价类的个数，
    并取等价类数量最大的元组属性的补集进行泛化，所泛化的属性值是元组中等价类对应的数据项中的属性值，
    将泛化后的结果返回到第一步进行K匿名检测
    '''
    equil_num_list = []
    attr_k_check_res_dict = {}
    for i in range(QID):
        tmp_qid = QID.copy()
        tmp_qid.remove(QID[i])
        attr_k_check_res = Test_K_Anonymity(tmp_data,tmp_qid,k)
        equil_num = sum(attr_k_check_res)      
        equil_num_list.append(equil_num)
        attr_k_check_res_dict[i] = attr_k_check_res

    '''
    选择待泛化的属性列，规则为：
    存在等价类数据多的列，若等价类数目一样，则选择属性取值多的那一个进行泛化
    '''
    equil_max = max(equil_num_list)
    selected_attr_num = 0
    attr_num = 0
    for i in range(equil_num_list):
        if equil_num_list[i] == equil_max:
            tmp_attr_num = len(set(tmp_data[QID[i]])) ## 如果等价类个数满足条件，则计算属性值类型个数
            if tmp_attr_num >= attr_num:
                selected_attr_num = i
   
    ## 对选择的属性列进行泛化，并重新运行整个过程
    generalize(tree_dict,gen_col, data, index_boolean):
    tmp_data = generalize(TreeDict, QID[selected_attr_num], tmp_data, attr_k_check_res_dict[selected_attr_num])
    
    print("泛化属性列为" + QID[selected_attr_num] + "泛化后的结果为：")
    print(tmp_data)
    
    

In [12]:
my_list = ['a','b','c']
my_list

['a', 'b', 'c']

In [14]:
your_list = my_list.copy()
your_list.remove('b')
print(my_list)
print(your_list)

['a', 'b', 'c']
['a', 'c']
